# Here we test whether main works at all


In [1]:
import main
main.main("between", 3, 0, 3, 0, 0)

100%|██████████| 3/3 [00:00<00:00, 42.25it/s]


,Gen,Price,Mismatch,Val_NT,Num_TF,Num_VI,Num_NT,Mean_TF,Mean_VI,Mean_NT,Div,RDiv,WShare_TF,WShare_VI,WShare_NT,Pos+,Pos-,Rep
0,0.0,993.049924,-1.148043e+06,10298.0,1.0,1.0,1.0,2.0,10000.0,10000.0,0.000039,-1.650184,33.333333,33.333333,33.333333,1500000.0,1.148043e+06,0.0
1,1.0,9876.624593,-3.613875e+05,10795.0,1.0,1.0,1.0,2.0,10000.0,10000.0,0.000039,-1.768310,33.333333,33.333333,33.333333,1500000.0,1.509430e+06,0.0
2,2.0,37640.044741,-1.226571e+05,12986.0,1.0,1.0,1.0,2.0,10000.0,10000.0,0.000039,1.254435,32.051481,33.974060,33.974459,1500000.0,1.509430e+06,0.0


# Here we test whether the trading signal and excess demand functions/values work well


In [2]:
import sampling
import balance_sheet as bs
pop = sampling.create_pop("between", 3)
print(pop)
price_history = [100, 100]
current_price = 100
bs.calculate_wealth(pop, current_price)
bs.determine_edf(pop, price_history)
# Under simulations for market clearing in Colab, we obtain that the clearing price is 5312.95
""" def func1(asset_key, price): #value investor
  return ((50_000_000 + 500_000 * float(initial_price)) / price) * (np.tanh(np.log2(10000) - np.log2(price)) + 0.5) - 500_000

def func2(asset_key, price): #noise trader
  return ((50_000_000 + 500_000 * float(initial_price)) / price) * (np.tanh(np.log2(10000) - np.log2(price)) + 0.5) - 500_000

def func3(asset_key, price): #trend follower
  return ((50_000_000 + 500_000 * float(initial_price)) / price) * (np.tanh(0) + 0.5) - 500_000

"""
current_price = 233
bs.calculate_edv(pop, current_price)

true_value_tf = -285711
true_value_vi = 142849
true_value_nt = 142849

for ind in pop:
    print(ind.type)
    print(ind.edv)

# Result: TF and VI are exact. NT is only marginally wrong (due to ind.process being 1 in the Colab reference)
# Hence: the EDV functions well.

print(bs.calculate_total_edv(pop))
# THe deviation is understandable with regards to the NT randomness factor.

[[2], [10000.0], [10000.0]]
tf
-285407.7253218884
vi
143760.11887624383
nt
143753.2902227376
2105.683777093014


# Testing whether wealth earnings profit can be imported


In [2]:
import balance_sheet as bs
import sampling
pop = sampling.create_pop("between", 3)
bs.wealth_earnings_profit(pop, 1, 10)

([[2], [10000.0], [10000.0]], 0.9977060218238474, -0.6173766483997932)

# Here we test whether market aggregate_edf works as expected

In [5]:
import sampling as sampling
import numpy as np
import balance_sheet as bs
pop = sampling.create_pop("between", 3)
price_history = []
current_price = 100
bs.calculate_wealth(pop, current_price)
bs.determine_edf(pop, price_history)

# Define aggregate demand function
def squared_agg_ed(p): 
    result = 0
    for ind in pop:
        result += ind.edf(p)
    return result

for ind in pop:
    if ind.type == "nt":
        process = ind.process
        print("Process identified " + str(process))

def func1(price): #value investor
  return ((50_000_000 + 500_000 * float(current_price)) / price) * (np.tanh(np.log2(10000) - np.log2(price)) + 0.5) - 500_000

def func2(price): #noise trader
  return ((50_000_000 + 500_000 * float(current_price)) / price) * (np.tanh(np.log2(10000 * process) - np.log2(price)) + 0.5) - 500_000

def func3(price): #trend follower
  return ((50_000_000 + 500_000 * float(current_price)) / price) * (np.tanh(0) + 0.5) - 500_000

print("Real result (1): " + str(func1(1) + func2(1) + func3(1)))
print("Function result (1): " + str(squared_agg_ed(1)) )

print("Real result (10): " + str(func1(10) + func2(10) + func3(10)))
print("Function result (10): " + str(squared_agg_ed(10)) )

# They are the same. squared_agg_ed in leap_mc works well.


Process identified 0.9873329030372872
Real result (1): 348499999.998829
Function result (1): 348499999.998829
Real result (10): 33499999.91006049
Function result (10): 33499999.91006049
